<a href="https://colab.research.google.com/github/er-mukeshmahato/Adafruit_SSD1306/blob/master/Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Upload kaggle.json (your Kaggle API token)


In [18]:
from google.colab import files
files.upload()  # Upload kaggle.json file here when prompted

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mukeshmahato1122","key":"79e16915a31ef978d4ff90118f80a710"}'}

# Step 2: Setup Kaggle credentials and install kaggle CLI

In [19]:
import os
import shutil

# Setup kaggle credentials
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 600)

# Step 3: Download dataset from Kaggle

In [20]:

!kaggle datasets download -d pranavgautam29/chest-x-ray-imagesmultiple-diseases



Dataset URL: https://www.kaggle.com/datasets/pranavgautam29/chest-x-ray-imagesmultiple-diseases
License(s): apache-2.0
chest-x-ray-imagesmultiple-diseases.zip: Skipping, found more recently modified local copy (use --force to force download)


# Step 4: Verify and unzip the dataset

In [21]:

!unzip -o -q /content/chest-x-ray-imagesmultiple-diseases.zip -d /content/data


# Step 5: Import libraries and set parameters

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks,regularizers
import matplotlib.pyplot as plt

data_dir = "/content/data/data"  # Path inside unzipped folder; adjust if necessary
batch_size = 32
img_height = 224
img_width = 224

epochs_initial = 10
epochs_finetune = 10


# Step 6: Load dataset with training and validation splits

In [23]:

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes found:", class_names)

Found 4452 files belonging to 5 classes.
Using 3562 files for training.
Found 4452 files belonging to 5 classes.
Using 890 files for validation.
Classes found: ['Covid', 'Normal', 'Pneumonia', 'Pneumothorax', 'Tuberculosis']


In [24]:
# Step 7: Define data augmentation and prepare datasets

In [25]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
    layers.RandomTranslation(0.1, 0.1),
    layers.GaussianNoise(0.05)
])

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


# Step 8: Build model using densenet with transfer learning

In [26]:
base_model = tf.keras.applications.DenseNet121(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base initially

inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.densenet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names),
                       activation='softmax',
                       kernel_regularizer=regularizers.l2(1e-4))(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_6 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add_3 (Add)                     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_7 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         5,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,042,629 (26.87 MB)

 Trainable params: 5,125 (20.02 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

# Step 9: Define callbacks


In [27]:
checkpoint_path = "best_model.keras"
cb = [
    callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True),
    callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
]

# Step 10: Initial training



In [28]:
# Initial training with base model frozen or partially frozen
history_initial = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_initial,  # shorter initial training
    callbacks=cb
)

# Unfreeze more layers for fine-tuning
for layer in base_model.layers[-60:]:
    layer.trainable = True

# Recompile with lower LR for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_finetune,
    callbacks=cb
)

model.save("refined_final_model.keras")
print("Refined model saved.")


Epoch 1/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 67s 298ms/step - accuracy: 0.2287 - loss: 2.3077 - val_accuracy: 0.2719 - val_loss: 1.5695
Epoch 2/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.2787 - loss: 1.9973

KeyboardInterrupt: 

# Step 11: Fine-tuning

In [ ]:

# Unfreeze top 60 layers
for layer in base_model.layers[-60:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_finetune,
    callbacks=cb
)

# Step 12: Save final model


In [ ]:
model.save("final_finetuned_model.keras")
print("Fine-tuned model saved.")

# Step 13: Plot training results

In [ ]:
acc = history_initial.history['accuracy'] + history_finetune.history['accuracy']
val_acc = history_initial.history['val_accuracy'] + history_finetune.history['val_accuracy']
loss = history_initial.history['loss'] + history_finetune.history['loss']
val_loss = history_initial.history['val_loss'] + history_finetune.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy', marker='o')
plt.plot(epochs_range, val_acc, label='Val Accuracy', marker='o')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss', marker='o')
plt.plot(epochs_range, val_loss, label='Val Loss', marker='o')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

What you can do next:
Fine-tune the model:
Unfreeze some top DenseNet layers to improve feature extraction on your dataset.

Use a lower learning rate during fine-tuning (e.g., 1e-4 or 1e-5) to avoid destroying pretrained weights.

Add stronger regularization or augmentations if validation accuracy plateaus or overfitting starts.
